In [2]:
import os
import numpy as np
import tensorflow as tf
import cv2

# Configurações
IMG_SIZE = 223

def preprocess_image(image_array):

    if len(image_array.shape) == 3 and image_array.shape[2] == 3:
        image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    image_array = image_array.astype(np.uint8)
    image_array = cv2.equalizeHist(image_array)
    image_array = cv2.GaussianBlur(image_array, (5, 5), 0)
    image_array = image_array.astype(np.float32) / 255.0
    image_array = cv2.resize(image_array, (IMG_SIZE, IMG_SIZE))
    image_array = np.expand_dims(image_array, axis=-1)
    image_array = np.expand_dims(image_array, axis=0)

    return image_array

def image_predict(model_path, preprocessed_image_array): # --> Para o pessoal do Back end
    modelo = tf.keras.models.load_model(model_path)
    print("Resumo do modelo carregado:")
    modelo.summary()
    predicao = modelo.predict(preprocessed_image_array)
    return predicao

def main():
    path_imagem_teste = '/content/test_image/1018.png'
    path_modelo = '/content/best_model (1).keras'
    if not os.path.exists(path_imagem_teste):
        print(f"Erro: Imagem de teste não encontrada em '{path_imagem_teste}'")
        return
    if not os.path.exists(path_modelo):
        print(f"Erro: Modelo não encontrado em '{path_modelo}'")
        return
    imagem_original = cv2.imread(path_imagem_teste, cv2.IMREAD_GRAYSCALE)

    if imagem_original is None:
        print(f"Erro ao carregar a imagem: {path_imagem_teste}")
        return
    imagem_preprocessada = preprocess_image(imagem_original)
    print(f"Shape da imagem após pré-processamento: {imagem_preprocessada.shape}")


    # Fazer a predição
    predicao_imagem = image_predict(path_modelo, imagem_preprocessada)
    if predicao_imagem is not None:
        print("\nPredição (valor bruto):", predicao_imagem)
        if predicao_imagem[0][0] > 0.5:
            print("Resultado: Não Fraturado (Classe 1)")
        else:
            print("Resultado: Fraturado (Classe 0)")

if __name__ == '__main__':
    main()

Shape da imagem após pré-processamento: (1, 223, 223, 1)
Resumo do modelo carregado:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 223, 223, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,505,541 (127.81 MB)

 Trainable params: 11,168,513 (42.60 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,337,028 (85.21 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 982ms/step

Predição (valor bruto): [[0.00209237]]
Resultado: Fraturado (Classe 0)
